In [1]:
import pandas as pd
import sys
import os
import json
from datetime import datetime

# Add src to path to import local modules
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

# Import your custom modules
from utils import ResourceTracker, set_global_seed, ExperimentLogger
from nlp_models import (
    load_zero_shot_pipeline, 
    extract_context_sentences, 
    classify_sentences_batch, 
    get_hf_model_metadata,
    get_accompanying_terms
)
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import numpy as np
from ast import literal_eval

# Initialize Logging and Seeding
logger = ExperimentLogger(log_dir="../reports")
set_global_seed(42)

Logging experiments to: ../reports/experiment_20260107_235551.json
Global seed set to: 42


# 1. Finding top countries

In [2]:
# Finding top countries
df = pd.read_csv("../data/processed/putins_talks_with_countries.csv")
all_countries = df['extracted_countries'].apply(literal_eval).explode().dropna()
country_counts = all_countries[all_countries != 'Russia'].value_counts()
country_counts

extracted_countries
Ukraine           3838
China             2964
Syria             2384
United States     1646
Turkey            1468
                  ... 
Greenland            1
Virgin Islands       1
Seychelles           1
Samoa                1
Bhutan               1
Name: count, Length: 170, dtype: int64

In [3]:
country_counts.to_csv("../data/processed/country_counts.csv")

In [4]:
top_countries = country_counts.head(5).index.tolist()
top_countries

['Ukraine', 'China', 'Syria', 'United States', 'Turkey']

# 2. Extracting sentences mentioning chosen countries

In [2]:
# Create target terms
target_terms = {
    'Poland': ['Poland', 'Polish', 'Poles'],
    'Ukraine': ['Ukraine', 'Ukrainian', 'Ukrainians'],
    'United States': ['United States', 'USA', 'American', 'Americans', 'U.S.'],
    'China': ['China', 'Chinese'],
    'Syria': ['Syria', 'Syrian'],
    'Russia': ['Russia', 'Russian', 'Russians', 'Soviets', 'Soviet Union', 'USSR', 'Soviet', 'Russian Federation']
}

In [8]:
top_countries.append('Poland')
for key, terms in target_terms.items():
    print(f"Processing country: {key}")
    # Extract context sentences
    with ResourceTracker("Sentence Extraction") as rt:
        extracted_data = extract_context_sentences(
            texts=df['transcript_filtered'].tolist(), 
            dates=df['date'].tolist(),
            target_terms=terms,
            spacy_model_name="en_core_web_sm"
        )
    logger.log_operation("Sentence_Extraction", rt.duration, rt.peak_memory_mb, {"count": len(extracted_data)})
    df_sentences = pd.DataFrame(extracted_data)
    print(f"Found {len(df_sentences)} relevant sentences.")
    df_sentences.to_csv(f"../data/sentences/sentences_{key}.csv", index=False)
    df_sentences.head()

Processing country: Poland
[Sentence Extraction] Finished.
   Duration: 258.9408 seconds
   Peak Memory: 527.33 MB
Found 361 relevant sentences.
Processing country: Ukraine
[Sentence Extraction] Finished.
   Duration: 255.7312 seconds
   Peak Memory: 525.70 MB
Found 3488 relevant sentences.
Processing country: United States
[Sentence Extraction] Finished.
   Duration: 255.0165 seconds
   Peak Memory: 525.84 MB
Found 3478 relevant sentences.
Processing country: China
[Sentence Extraction] Finished.
   Duration: 255.6271 seconds
   Peak Memory: 525.72 MB
Found 2721 relevant sentences.
Processing country: Syria
[Sentence Extraction] Finished.
   Duration: 253.4403 seconds
   Peak Memory: 536.38 MB
Found 2213 relevant sentences.
Processing country: Russia
[Sentence Extraction] Finished.
   Duration: 254.2516 seconds
   Peak Memory: 526.76 MB
Found 32359 relevant sentences.


# 3. Finding accompanying terms

In [6]:
for key, terms in target_terms.items():
    print(f"Processing country: {key}")
    df_sentences = pd.read_csv(f"../data/sentences/sentences_{key}.csv")
    all_accompanying_terms = []
    for sentence in df_sentences['sentence']:
        accompanying_terms = get_accompanying_terms(
            sentence, 
            terms
        )
        all_accompanying_terms.append(accompanying_terms)
    df_sentences['accompanying_terms'] = all_accompanying_terms
    df_sentences.to_csv(f"../data/sentences/sentences_{key}_with_terms.csv", index=False)

Processing country: Poland
Processing country: Ukraine
Processing country: United States
Processing country: China
Processing country: Syria
Processing country: Russia


# 4. Classifying sentences using zero-shot classification

In [7]:
# Configuration
LABELS_BASE = ["partner", "enemy", "neutral"]
LABELS_SYNONYMS = ["ally", "adversary", "unbiased"]

# Load Model
model_name = "facebook/bart-large-mnli"
print(f"Loading model: {model_name}...")
zs_pipeline = load_zero_shot_pipeline(model_name)

# Log Model Metadata for Report
meta = get_hf_model_metadata(zs_pipeline)
print(f"Model Architecture: {meta['model_architecture']}")
print(f"Commit Hash: {meta['commit_hash']}")

Loading model: facebook/bart-large-mnli...


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Model Architecture: BartForSequenceClassification
Commit Hash: d7645e127eaf1aefc7862fd59a17a5aa8558b8ce


In [12]:
for key, terms in target_terms.items():
    print(f"Classifying sentences for country: {key}")
    df_sentences = pd.read_csv(f"../data/sentences/sentences_{key}_with_terms.csv")
    
    if key == 'Russia':
        LABELS_BASE = ["victim", "leader", "defender"]
        LABELS_SYNONYMS = ["casualty", "commander", "protector"]
    
    # Classify sentences in batches
    print("Running Zero-Shot (Base Labels)...")
    with ResourceTracker("ZeroShot_BaseLabels") as rt:
        results_base = classify_sentences_batch(
            zs_pipeline, 
            df_sentences['sentence'].tolist(), 
            LABELS_BASE
        )
    
    logger.log_operation(
        name="ZeroShot_Base",
        duration=rt.duration,
        memory_mb=rt.peak_memory_mb,
        metrics={"model": model_name, "labels": LABELS_BASE, "hash": meta['commit_hash']}
    )
    
    df_sentences['zs_base_label'] = [r['top_label'] for r in results_base]
    df_sentences['zs_base_score'] = [r['top_score'] for r in results_base]
    
    print("Running Zero-Shot (Synonym Labels)...")
    with ResourceTracker("ZeroShot_SynonymLabels") as rt:
        results_syn = classify_sentences_batch(
            zs_pipeline, 
            df_sentences['sentence'].tolist(), 
            LABELS_SYNONYMS)
    
    logger.log_operation(
        name="ZeroShot_Synonyms",
        duration=rt.duration,
        memory_mb=rt.peak_memory_mb,
        metrics={"model": model_name, "labels": LABELS_SYNONYMS}
    )
    
    df_sentences['zs_syn_label'] = [r['top_label'] for r in results_syn]
    df_sentences['zs_syn_score'] = [r['top_score'] for r in results_syn]
    
    # Save classified results
    df_sentences.to_csv(f"../data/sentences/sentences_{key}_classified.csv", index=False)
    print(f"Saved classified sentences for {key}.")

Classifying sentences for country: Poland
Running Zero-Shot (Base Labels)...
[ZeroShot_BaseLabels] Finished.
   Duration: 45.8916 seconds
   Peak Memory: 0.77 MB
Running Zero-Shot (Synonym Labels)...
[ZeroShot_SynonymLabels] Finished.
   Duration: 45.2267 seconds
   Peak Memory: 0.35 MB
Saved classified sentences for Poland.
Classifying sentences for country: Ukraine
Running Zero-Shot (Base Labels)...
[ZeroShot_BaseLabels] Finished.
   Duration: 415.3089 seconds
   Peak Memory: 3.53 MB
Running Zero-Shot (Synonym Labels)...
[ZeroShot_SynonymLabels] Finished.
   Duration: 414.4275 seconds
   Peak Memory: 3.36 MB
Saved classified sentences for Ukraine.
Classifying sentences for country: United States
Running Zero-Shot (Base Labels)...
[ZeroShot_BaseLabels] Finished.
   Duration: 412.3324 seconds
   Peak Memory: 3.49 MB
Running Zero-Shot (Synonym Labels)...
[ZeroShot_SynonymLabels] Finished.
   Duration: 412.1403 seconds
   Peak Memory: 3.35 MB
Saved classified sentences for United States.

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


[ZeroShot_BaseLabels] Finished.
   Duration: 263.4440 seconds
   Peak Memory: 2.21 MB
Running Zero-Shot (Synonym Labels)...
[ZeroShot_SynonymLabels] Finished.
   Duration: 263.1236 seconds
   Peak Memory: 2.14 MB
Saved classified sentences for Syria.
Classifying sentences for country: Russia
Running Zero-Shot (Base Labels)...
[ZeroShot_BaseLabels] Finished.
   Duration: 3792.1597 seconds
   Peak Memory: 33.50 MB
Running Zero-Shot (Synonym Labels)...
[ZeroShot_SynonymLabels] Finished.
   Duration: 3792.4268 seconds
   Peak Memory: 31.09 MB
Saved classified sentences for Russia.


# 5. Classifying sentences using Gemini and human annotation

# 6. Classification evaluation

# 7. Aggregating and saving classification counts

In [ ]:
def aggregate_counts(df, method_col, method_name):
    """Counts occurrences of each label per country for a given method."""
    # Group by Country (found_term) and Label
    counts = df.groupby(['found_term', method_col]).size().reset_index(name='count')
    counts['method'] = method_name
    counts.rename(columns={method_col: 'label'}, inplace=True)
    return counts

In [ ]:
# List of method columns to aggregate
methods_to_count = [
    ('zs_base_label', 'ZeroShot_Base'),
    ('zs_syn_label', 'ZeroShot_Synonym'),
    ('ai_prompt_a', 'AI_Prompt_A'),
    ('ai_prompt_b', 'AI_Prompt_B')
]

all_counts = []

for col, name in methods_to_count:
    if col in df_sentences.columns:
        all_counts.append(aggregate_counts(df_sentences, col, name))

# Combine all into one DataFrame
if all_counts:
    df_counts = pd.concat(all_counts, ignore_index=True)
    
    # Reorder for clarity: Country, Method, Label, Count
    df_counts = df_counts[['found_term', 'method', 'label', 'count']]
    
    print("Summary of Classification Counts:")
    display(df_counts.head(15))
    
    # Save
    df_counts.to_csv("../data/processed/country_label_counts.csv", index=False)
else:
    print("No classification columns found to aggregate.")